<div class='alert alert-danger'>
    <h1>I. Web Scrapping</h1>
    <p>The objective of this project is to get the information of laptop from a computer store</p>
</div>

<div class='alert alert-info'>
    <h3>Topic: Scrap Laptop</h3>
    <p>Team: 3 Thim, Vanda, Dara</p>
</div>

In [205]:
#import libraries
import re
import csv
import random
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import numpy as np
%matplotlib inline

In [206]:
#Defind web page url

url = 'https://www.goldonecomputer.com/index.php?route=product/category&path=58&limit=100'
url2 = 'https://www.goldonecomputer.com/index.php?route=product/category&path=58&limit=100&page=2'
url3 = 'https://www.goldonecomputer.com/index.php?route=product/category&path=58&limit=100&page=3'
url4 = 'https://www.goldonecomputer.com/index.php?route=product/category&path=58&limit=100&page=4'
url5 = 'https://tkcustomcomputer.com/menu/laptop'
url6 = 'https://ptc-computer.com.kh/product-category/computer-computer/laptops-computer/'

In [207]:
#add agent to pass throw restriction

user_agents = [
  "Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0",
  "Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0",
  "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0"
  ]

random_user_agent = random.choice(user_agents)
headers = {
    'User-Agent': random_user_agent
}

In [208]:
#start requesting page
response1 = requests.get(url2, headers=headers)
response2 = requests.get(url2, headers=headers)
response3 = requests.get(url3, headers=headers)
response4 = requests.get(url4, headers=headers)
response5 = requests.get(url5, headers=headers)
response6 = requests.get(url6,headers=headers)

In [209]:
#parse to beautiful soup
soup1 = BeautifulSoup(response1.text, 'html.parser')
soup2 = BeautifulSoup(response2.text, 'html.parser')
soup3 = BeautifulSoup(response3.text, 'html.parser')
soup4 = BeautifulSoup(response4.text, 'html.parser')
soup5 = BeautifulSoup(response5.text, 'html.parser')
soup6 = BeautifulSoup(response6.text, 'html.parser')

In [210]:
#find product block by Div element with element product-category

content1 = soup1.find('div',id='product-category').find('div',id='content',class_='col-sm-9')
content2 = soup2.find('div',id='product-category').find('div',id='content',class_='col-sm-9')
content3 = soup3.find('div',id='product-category').find('div',id='content',class_='col-sm-9')
content4 = soup4.find('div',id='product-category').find('div',id='content',class_='col-sm-9')
content5 = soup5.find('div',class_='content-right')
content6 = soup6.find('div',class_='wcan-products-container')

In [211]:
#method

#find price
def get_price(pro):
    price_old = pro.find('p',class_='price') #normal price
    price_new = pro.find('span',class_='price-new') # discount price
    if price_new != None:
        price_old = price_new
    return price_old.text.strip()

#extract detail as list
def extract_detail(pro):
    detail = pro.find('h4')
    index_start = detail.text.index('(') + 1
    index_end = detail.text.index(')')
    detail = detail.text[index_start:index_end]  
    return detail.split("/")

#get cpu
def get_cpu(pro):
    return extract_detail(pro)[0]

#get ram
def get_ram(pro):
    return re.findall('[0-9]+', extract_detail(pro)[1])[0]


#get hard disk
def get_harddisk(pro):
    hd = re.findall('[\d]+',extract_detail(pro)[2])[0]
    if len(hd) == 1 :
        hd = int(hd)*1024
    return hd

#get other
def get_other(pro):
    result = ""
    if len(extract_detail(pro)) > 1:
        result = extract_detail(pro)[3]
    
    return result

#get cup
def get_cpu_tk(pro):
    desc = pro.find('div',class_='pro-descript').text
    arr = re.findall(r'CPU : (.*?)\-', desc, re.DOTALL)
    info = "None"
    if len(arr) == 1:
        info = arr[0]

    return info

#get ram
def get_ram_tk(pro):
    desc = pro.find('div',class_='pro-descript').text
    arr = re.findall(r'RAM : (.*?)\-', desc, re.DOTALL)
    if len(arr) == 1:
        return re.findall('[0-9]+',arr[0])[0]
    
    return 0

#get ram
def get_hdd_tk(pro):
    desc = pro.find('div',class_='pro-descript').text
    arr = re.findall(r'Storage : (.*?)\-', desc, re.DOTALL)
    
    if len(arr) == 1:
        hd = re.findall('[0-9]+',arr[0])[0]
        if len(hd) == 1 :
            hd = int(hd)*1024
        return hd
    
    return 0

#get hhd tk
def get_hdd_tks(pro):
    desc = pro.find('div',class_='pro-descript').text
    arr = re.findall(r'Storage : (.*?)\-', desc, re.DOTALL)
    info = "None"
    if len(arr) == 1:
        info = arr[0]

    return info

In [212]:
laptop_list = []
products = content1.find_all('div', class_='product-layout') 

for pro in products:
    title = pro.find('h4')
    price = get_price(pro)
    name = title.text.split("(",1)[0]
    cpu = get_cpu(pro)
    ram = get_ram(pro)
    hdd = get_harddisk(pro)
    
   
    d = {}
    d['Price'] = price.replace('$','')
    d['Name'] = name
    d['CPU'] = cpu
    d['RAM'] = ram
    d['HardDisk'] = hdd
    
    laptop_list.append(d)

In [213]:
products2 = content2.find_all('div', class_='product-layout') 


for pro in products2:
    title = pro.find('h4')
    price = get_price(pro)
    name = title.text.split("(",1)[0]
    cpu = get_cpu(pro)
    ram = get_ram(pro)
    hdd = get_harddisk(pro)
    oth = get_other(pro)
    
    #Write to file
    d = {}
    d['Price'] = price.replace('$','')
    d['Name'] = name
    d['CPU'] = cpu
    d['RAM'] = ram
    d['HardDisk'] = hdd
    

    laptop_list.append(d)

In [214]:
products3 = content3.find_all('div', class_='product-layout') 


for pro in products3:
    title = pro.find('h4')
    price = get_price(pro)
    name = title.text.split("(",1)[0]
    cpu = get_cpu(pro)
    ram = get_ram(pro)
    hdd = get_harddisk(pro)

    
    d = {}
    d['Price'] = price.replace('$','')
    d['Name'] = name
    d['CPU'] = cpu
    d['RAM'] = ram
    d['HardDisk'] = hdd
    

    laptop_list.append(d)

In [215]:
products4 = content4.find_all('div', class_='product-layout') 


for pro in products4:
    title = pro.find('h4')
    price = get_price(pro)
    name = title.text.split("(",1)[0]
    cpu = get_cpu(pro)
    ram = get_ram(pro)
    hdd = get_harddisk(pro)
    
    
    d = {}
    d['Price'] = price.replace('$','')
    d['Name'] = name
    d['CPU'] = cpu
    d['RAM'] = ram
    d['HardDisk'] = hdd
    

    laptop_list.append(d)

In [227]:
#TK computer
products5 = content5.find_all('div',class_='card-body') 

for pro in products5:
    name = pro.find('h6',class_='card-title').text
    price = pro.find('h4',class_='price').text.replace('$','')
    cpu = get_cpu_tk(pro)
    ram = get_ram_tk(pro)
    hdd = get_hdd_tk(pro)
       
    d = {}
    d['Price'] = price
    d['Name'] = name
    d['CPU'] = cpu
    d['RAM'] =  ram
    d['HardDisk'] = hdd
    
    laptop_list.append(d)

In [228]:
#white file
#filename = 'laptop_20230708.csv'
#with open(filename,'w',newline='') as f:
#    w = csv.DictWriter(f,['Price','Name','CPU','RAM','HardDisk'])
#    w.writeheader()       
#    w.writerows(laptop_list)

In [288]:
df = pd.DataFrame(laptop_list)
df
#df
#df['Price'] = df['Price'].astype(float)
#df.info()
#df.info()

,Price,Name,CPU,RAM,HardDisk
0,279.00,Asus Vivobook X415MA-BV754W,Celeron® N4020,4,256
1,289.00,Asus Vivobook X515MA-BR902W,Celeron N4020,4,256
2,949.00,Asus Zenbook 14 UX3402ZA-KP168W,i5 1240P,8,512
3,"1,299.00",Asus ZenBook 14 Flip OLED UP3404VA-KN058W,I7 1360P,16,1024
4,949.00,Asus Zenbook 14 OLED UM3402YA-KM454W,R7 7730U,16,1024
...,...,...,...,...,...
480,499.00,,Intel® Core™ i3,4,512
481,499.00,MSI Modern 14 C12M,Intel® Core™ i3,4,512
482,769.00,IdeaPad Gaming 3 15ARH7,None,8,512
483,"1,639.00",Legion Slim 5 16IRH8,None,16,512
